In [99]:
import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
import datetime
import xlrd

path='./'
fire_xlsx=pd.read_excel(path+'서울시 화재 출동현황(2018.1_2019.11).xlsx')

# '시도' 칼럼 삭제
fire_xlsx.drop('시도', axis=1, inplace=True)

display(fire_xlsx)

,소방서,119안전센터,접수일시,출동일시,도착일시,시군구,읍면동,소방서거리(㎞)
0,강북소방서,번동119안전센터,2018-01-01 00:49:38,2018-01-01 00:50:34,2018-01-01 00:55:00,강북구,번동,1.4
1,관악소방서,봉천119안전센터,2018-01-01 02:34:10,2018-01-01 02:36:17,2018-01-01 02:39:00,관악구,봉천동,0.8
2,강북소방서,번동119안전센터,2018-01-01 05:04:43,2018-01-01 05:05:53,2018-01-01 05:10:00,강북구,수유동,1.7
3,구로소방서,시흥119안전센터,2018-01-01 08:09:23,2018-01-01 08:10:00,2018-01-01 08:16:00,금천구,시흥동,1.5
4,구로소방서,독산119안전센터,2018-01-01 08:20:13,2018-01-01 08:20:40,2018-01-01 08:25:00,금천구,시흥동,1.9
...,...,...,...,...,...,...,...,...
11490,구로소방서,공단119안전센터,2019-11-30 15:21:13,2019-11-30 15:22:04,2019-11-30 15:25:59,구로구,가리봉동,1.4
11491,관악소방서,관악119안전센터,2019-11-30 17:59:03,2019-11-30 18:00:44,2019-11-30 18:04:00,관악구,봉천동,0.5
11492,구로소방서,공단119안전센터,2019-11-30 20:43:20,2019-11-30 20:44:41,2019-11-30 20:49:56,금천구,가산동,2.4
11493,성북소방서,돈암119안전센터,2019-11-30 23:10:00,2019-11-30 23:11:46,2019-11-30 23:13:00,성북구,동선동5가,0.8


In [100]:
fire_xlsx.columns = ["FirestationParent", "FirestationChild", "ReceiptTime", "DepartureTime", "ArrivalTime", "Town", "Village", "DistanceKm"]

display(fire_xlsx)

,FirestationParent,FirestationChild,ReceiptTime,DepartureTime,ArrivalTime,Town,Village,DistanceKm
0,강북소방서,번동119안전센터,2018-01-01 00:49:38,2018-01-01 00:50:34,2018-01-01 00:55:00,강북구,번동,1.4
1,관악소방서,봉천119안전센터,2018-01-01 02:34:10,2018-01-01 02:36:17,2018-01-01 02:39:00,관악구,봉천동,0.8
2,강북소방서,번동119안전센터,2018-01-01 05:04:43,2018-01-01 05:05:53,2018-01-01 05:10:00,강북구,수유동,1.7
3,구로소방서,시흥119안전센터,2018-01-01 08:09:23,2018-01-01 08:10:00,2018-01-01 08:16:00,금천구,시흥동,1.5
4,구로소방서,독산119안전센터,2018-01-01 08:20:13,2018-01-01 08:20:40,2018-01-01 08:25:00,금천구,시흥동,1.9
...,...,...,...,...,...,...,...,...
11490,구로소방서,공단119안전센터,2019-11-30 15:21:13,2019-11-30 15:22:04,2019-11-30 15:25:59,구로구,가리봉동,1.4
11491,관악소방서,관악119안전센터,2019-11-30 17:59:03,2019-11-30 18:00:44,2019-11-30 18:04:00,관악구,봉천동,0.5
11492,구로소방서,공단119안전센터,2019-11-30 20:43:20,2019-11-30 20:44:41,2019-11-30 20:49:56,금천구,가산동,2.4
11493,성북소방서,돈암119안전센터,2019-11-30 23:10:00,2019-11-30 23:11:46,2019-11-30 23:13:00,성북구,동선동5가,0.8


In [101]:
# '출동요일' 칼럼 추가
day = []
for idx, row in fire_xlsx.iterrows():
    weekday = (row['ReceiptTime'].weekday())
    d = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    day.append(d[weekday])

fire_xlsx.loc[:, 'day'] = day
display(fire_xlsx)

,FirestationParent,FirestationChild,ReceiptTime,DepartureTime,ArrivalTime,Town,Village,DistanceKm,day
0,강북소방서,번동119안전센터,2018-01-01 00:49:38,2018-01-01 00:50:34,2018-01-01 00:55:00,강북구,번동,1.4,Monday
1,관악소방서,봉천119안전센터,2018-01-01 02:34:10,2018-01-01 02:36:17,2018-01-01 02:39:00,관악구,봉천동,0.8,Monday
2,강북소방서,번동119안전센터,2018-01-01 05:04:43,2018-01-01 05:05:53,2018-01-01 05:10:00,강북구,수유동,1.7,Monday
3,구로소방서,시흥119안전센터,2018-01-01 08:09:23,2018-01-01 08:10:00,2018-01-01 08:16:00,금천구,시흥동,1.5,Monday
4,구로소방서,독산119안전센터,2018-01-01 08:20:13,2018-01-01 08:20:40,2018-01-01 08:25:00,금천구,시흥동,1.9,Monday
...,...,...,...,...,...,...,...,...,...
11490,구로소방서,공단119안전센터,2019-11-30 15:21:13,2019-11-30 15:22:04,2019-11-30 15:25:59,구로구,가리봉동,1.4,Saturday
11491,관악소방서,관악119안전센터,2019-11-30 17:59:03,2019-11-30 18:00:44,2019-11-30 18:04:00,관악구,봉천동,0.5,Saturday
11492,구로소방서,공단119안전센터,2019-11-30 20:43:20,2019-11-30 20:44:41,2019-11-30 20:49:56,금천구,가산동,2.4,Saturday
11493,성북소방서,돈암119안전센터,2019-11-30 23:10:00,2019-11-30 23:11:46,2019-11-30 23:13:00,성북구,동선동5가,0.8,Saturday


In [102]:
# 출동시간=도착일시-출동일시 칼럼 추가(label)
label=[]

for idx, row in fire_xlsx.iterrows():
    diff=(row["ArrivalTime"]-row['DepartureTime']).seconds
    label.append(diff)

fire_xlsx.loc[:, 'TimeTakenSec'] = label # index = fire_xlsx.index
display(fire_xlsx)

,FirestationParent,FirestationChild,ReceiptTime,DepartureTime,ArrivalTime,Town,Village,DistanceKm,day,TimeTakenSec
0,강북소방서,번동119안전센터,2018-01-01 00:49:38,2018-01-01 00:50:34,2018-01-01 00:55:00,강북구,번동,1.4,Monday,266
1,관악소방서,봉천119안전센터,2018-01-01 02:34:10,2018-01-01 02:36:17,2018-01-01 02:39:00,관악구,봉천동,0.8,Monday,163
2,강북소방서,번동119안전센터,2018-01-01 05:04:43,2018-01-01 05:05:53,2018-01-01 05:10:00,강북구,수유동,1.7,Monday,247
3,구로소방서,시흥119안전센터,2018-01-01 08:09:23,2018-01-01 08:10:00,2018-01-01 08:16:00,금천구,시흥동,1.5,Monday,360
4,구로소방서,독산119안전센터,2018-01-01 08:20:13,2018-01-01 08:20:40,2018-01-01 08:25:00,금천구,시흥동,1.9,Monday,260
...,...,...,...,...,...,...,...,...,...,...
11490,구로소방서,공단119안전센터,2019-11-30 15:21:13,2019-11-30 15:22:04,2019-11-30 15:25:59,구로구,가리봉동,1.4,Saturday,235
11491,관악소방서,관악119안전센터,2019-11-30 17:59:03,2019-11-30 18:00:44,2019-11-30 18:04:00,관악구,봉천동,0.5,Saturday,196
11492,구로소방서,공단119안전센터,2019-11-30 20:43:20,2019-11-30 20:44:41,2019-11-30 20:49:56,금천구,가산동,2.4,Saturday,315
11493,성북소방서,돈암119안전센터,2019-11-30 23:10:00,2019-11-30 23:11:46,2019-11-30 23:13:00,성북구,동선동5가,0.8,Saturday,74


In [103]:
# '접수일시', '출동일시', '도착일시' 칼럼 삭제
fire_xlsx.drop("ReceiptTime", axis=1, inplace=True)
fire_xlsx.drop("DepartureTime", axis=1, inplace=True)
fire_xlsx.drop("ArrivalTime", axis=1, inplace=True)

display(fire_xlsx)

,FirestationParent,FirestationChild,Town,Village,DistanceKm,day,TimeTakenSec
0,강북소방서,번동119안전센터,강북구,번동,1.4,Monday,266
1,관악소방서,봉천119안전센터,관악구,봉천동,0.8,Monday,163
2,강북소방서,번동119안전센터,강북구,수유동,1.7,Monday,247
3,구로소방서,시흥119안전센터,금천구,시흥동,1.5,Monday,360
4,구로소방서,독산119안전센터,금천구,시흥동,1.9,Monday,260
...,...,...,...,...,...,...,...
11490,구로소방서,공단119안전센터,구로구,가리봉동,1.4,Saturday,235
11491,관악소방서,관악119안전센터,관악구,봉천동,0.5,Saturday,196
11492,구로소방서,공단119안전센터,금천구,가산동,2.4,Saturday,315
11493,성북소방서,돈암119안전센터,성북구,동선동5가,0.8,Saturday,74


In [104]:
fire_xlsx.to_csv("firestation_TimeTaken.csv", mode='w', encoding='utf-8')

In [105]:
# 학습시키기
# 콘솔창에 입력
# ludwig train --data_csv C:\\Users\\Playdata\\Desktop\\Safety_Project\\FireStation_PredictingTime\\firestation_TimeTaken.csv --model_definition "{input_features: [{name: FirestationParent, type: category}, {name: FirestationChild, type: category},{name: Town, type: category},{name: Village, type: category},{name: DistanceKm, type: numerical},{name: day, type: category}], output_features: [{name: TimeTakenSec, type: numerical}]}"

In [ ]:
# 가시화 
# 콘솔창에 입력
# ludwig visualize --visualization learning_curves --training_statistics C:\\Users\\Playdata\\Desktop\\Safety_Project\\FireStation_PredictingTime\\results\\experiment_run_1\\description.json